In [ ]:
import anthropic
import os
import base64
from anapath.params import *
from PIL import Image
import io

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.getenv("ANTHROPIC_API_KEY"),
)

def encode_image(image_path, max_width=512, max_height=256, quality=100):
    """
    Redimensionne l'image aux dimensions maximales spécifiées tout en conservant le ratio,
    puis l'encode en base64.
    """
    # Ouvrir l'image
    with Image.open(image_path) as img:
        # Préserver le ratio d'aspect
        width, height = img.size
        if width > max_width or height > max_height:
            ratio = min(max_width/width, max_height/height)
            new_width = int(width * ratio)
            new_height = int(height * ratio)
            img = img.resize((new_width, new_height), Image.LANCZOS)

        # Convertir en JPEG et encoder en base64
        buffer = io.BytesIO()
        img.convert('RGB').save(buffer, format="JPEG", quality=quality)
        buffer.seek(0)
        return base64.b64encode(buffer.read()).decode('utf-8')

# Chemin vers votre image
image_path = f"{train_tumor_path}/tile_02104_c78_r0_c0.png"

# Encodage de l'image
base64_image = encode_image(image_path)

# Replace placeholders like {{IMAGE_DESCRIPTION}} with real values,
# because the SDK does not support variables.
message = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=20000,
    temperature=1,
    system="You are an expert pathologist specializing in breast tissue analysis. Your task is to examine a digital image of a breast tissue sample and provide a detailed analysis, including a diagnosis of whether cancer is present or not.\n",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "You are an expert pathologist specializing in breast tissue analysis. Your task is to examine a digital image of a breast tissue sample and provide a detailed analysis, including a diagnosis of whether cancer is present or not.\n\nFirst, you will be given a description of the microscopic image of the breast tissue sample:\n\n<image_description>"
                },
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": base64_image
                    }
                },
                {
                    "type": "text",
                    "text": "Réponds à cette question en formatant ta réponse exclusivement en JSON valide, sans aucun texte avant ou après. Inclus les champs <analysis>, <diagnosis>, <confidence_level>, and <recommendations> tags. je veux que tu réponde en francais. Voici ce que tu dois faire :  {{IMAGE_DESCRIPTION}}\n</image_description>\n\nNext, you will receive relevant patient information:\n\n<patient_info>\n{{PATIENT_INFO}}\n</patient_info>\n\nCarefully analyze the image description and patient information. Consider the following aspects in your analysis:\n1. Cellular structure and organization\n2. Nuclear characteristics (size, shape, chromatin pattern)\n3. Presence of abnormal cell divisions or mitotic figures\n4. Tissue architecture and any disruptions\n5. Presence of inflammatory cells or other abnormal features\n\nBased on your analysis, determine whether the sample indicates the presence of breast cancer or not. \n\nProvide your analysis and diagnosis in the following format:\n\n<analysis>\n[Detailed description of your observations and interpretation of the tissue sample]\n</analysis>\n\n<diagnosis>\n[Clear statement of whether breast cancer is present or not]\n</diagnosis>\n\n<confidence_level>\n[Your level of confidence in the diagnosis on a scale of 1-10, where 1 is least confident and 10 is most confident]\n</confidence_level>\n\n<recommendations>\n[Any recommendations for further tests or follow-up procedures, if necessary]\n</recommendations>\n\nRemember, your final output should only include the content within the <analysis>, <diagnosis>, <confidence_level>, and <recommendations> tags. Do not include any additional commentary or thought processes outside of these specified sections."
                }
            ]
        }
    ],
    thinking={
        "type": "enabled",
        "budget_tokens": 16000
    }
)
print(message.content)

/tmp/ipykernel_16166/3372358609.py:26: DeprecationWarning: LANCZOS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((new_width, new_height), Image.LANCZOS)


OverloadedError: Error code: 529 - {'type': 'error', 'error': {'type': 'overloaded_error', 'message': 'Overloaded'}}

In [15]:
message.content[1].text

"# Breast Tissue Pathology Report\n\n<analysis>\nThe image shows a breast tissue sample with H&E (Hematoxylin and Eosin) staining at medium-high magnification. The specimen demonstrates significant histopathological abnormalities:\n\n- Cellular architecture: There is a marked disruption of normal breast tissue architecture with densely packed cellular areas arranged in solid sheets and nests, replacing normal glandular structures.\n\n- Cellular characteristics: The cells exhibit significant pleomorphism (variation in cell size and shape) with enlarged nuclei and prominent nucleoli.\n\n- Nuclear features: The nuclei are hyperchromatic (darkly stained), demonstrate irregular contours, and show increased nuclear-to-cytoplasmic ratios—all classic features of malignant transformation.\n\n- Mitotic activity: Increased mitotic figures are visible throughout the sample, suggesting high proliferative activity.\n\n- Invasion pattern: The abnormal cells appear to infiltrate the surrounding stroma